In [1]:
# third-party
import pandas as pd
import scipy
import pickle

# local
from get_data import transform_overview_on_target, transform_overview_on_overall
from performance import get_breath_parameters_performance
from delay_performance import normality_test
from performance import bland_altman_plot


In [2]:
with open('Results/results.pickle', 'rb') as file:
    overview = pickle.load(file)

overview_middle = {}
for id in overview.keys():
    overview_middle[id] = {key: value for key, value in overview[id].items() if key in ['SNBm', 'UALm', 'UARm']}

# remove middle activities
for id in overview.keys():
    del overview[id]["SNBm"]
    del overview[id]["UALm"]
    del overview[id]["UARm"]

### Activity-specific only RP analysis

In [3]:
# Transform overview from participant specific into all participants

overview_all_participants = transform_overview_on_target(overview, target="Activity")
overview_all_participants.keys()

dict_keys(['SNB', 'SGB', 'MIXB', 'STNB', 'MCH', 'SQT', 'AAL', 'AAR', 'ALL', 'ALR', 'UAL', 'UAR', 'SE', 'SS', 'TR'])

In [4]:
performance_all_participants = get_breath_parameters_performance(overview_all_participants, target="Activity")
performance_all_participants

,Activity,Sensor,MAE Ti (s),MRE Ti (%),MAE Te (s),MRE Te (%),MAE Tb (s),MRE Tb (%)
0,SNB,MAG,"0.17 , 0.28","8.43 , 13.26","0.18 , 0.33","7.80 , 11.88","0.16 , 0.32","3.40 , 5.43"
1,SNB,PZT,"0.37 , 0.50","15.62 , 22.19","0.38 , 0.53","12.94 , 18.33","0.29 , 0.45","4.64 , 7.01"
2,SGB,MAG,"0.81 , 0.64","17.37 , 14.85","0.69 , 0.66","11.85 , 11.26","0.64 , 0.67","5.57 , 5.62"
3,SGB,PZT,"1.57 , 1.30","35.82 , 30.03","1.64 , 1.31","26.41 , 20.92","0.84 , 0.97","6.97 , 7.78"
4,MIXB,MAG,"0.19 , 0.19","7.51 , 7.52","0.18 , 0.17","6.25 , 5.74","0.20 , 0.21","3.51 , 3.28"
5,MIXB,PZT,"0.56 , 0.72","17.27 , 22.19","0.64 , 0.87","16.58 , 25.04","0.65 , 0.96","8.14 , 11.90"
6,STNB,MAG,"0.16 , 0.24","7.30 , 9.85","0.19 , 0.29","7.46 , 10.80","0.13 , 0.22","2.48 , 2.69"
7,STNB,PZT,"0.29 , 0.44","10.53 , 15.44","0.35 , 0.56","10.87 , 18.98","0.26 , 0.48","4.02 , 7.54"
8,MCH,MAG,"0.31 , 0.36","14.31 , 14.77","0.33 , 0.36","15.89 , 15.80","0.33 , 0.42","7.13 , 8.29"
9,MCH,PZT,"0.63 , 0.60","28.61 , 24.60","0.67 , 0.64","30.65 , 27.90","0.62 , 0.65","13.03 , 13.55"


In [5]:
# mag = performance_all_participants[performance_all_participants["Sensor"] == "MAG"].loc[:,["SSE Ti", "SSE Te", "SSE Tb"]].reset_index(drop=True) 
# pzt = performance_all_participants[performance_all_participants["Sensor"] == "PZT"].loc[:,["SSE Ti", "SSE Te", "SSE Tb"]].reset_index(drop=True)

# a = mag / pzt 
# a["Activity"] = performance_all_participants[performance_all_participants["Sensor"] == "MAG"].reset_index(drop=True)["Activity"]
# a

### Overall RP analysis

In [6]:
# Transform overview from participant specific into all participants
overview_all = transform_overview_on_overall(overview)
overview_all.keys()

dict_keys(['MAG', 'Airflow', 'PZT'])

In [7]:
get_breath_parameters_performance(overview_all, target=None)

N=2814 breaths for MAG
N=2299 breaths for PZT


,Parameter,Sensor,Abs. error (s),Rel. error (%)
0,tI,MAG,"0.29 , 0.36","13.91 , 15.85"
1,tI,PZT,"0.57 , 0.59","25.75 , 23.83"
2,tE,MAG,"0.30 , 0.37","13.82 , 15.30"
3,tE,PZT,"0.59 , 0.61","26.03 , 24.12"
4,tB,MAG,"0.26 , 0.38","5.45 , 7.08"
5,tB,PZT,"0.48 , 0.59","10.16 , 11.64"


In [8]:
rp_detection_all = get_breath_parameters_performance(overview, target="both")
rp_detection_all

,ID,Activity,Sensor,MAE Ti (s),MRE Ti (%),MAE Te (s),MRE Te (%),MAE Tb (s),MRE Tb (%)
0,7OYX,SNB,MAG,"0.10 , 0.08","5.60 , 6.11","0.12 , 0.08","5.16 , 3.40","0.07 , 0.07","1.50 , 1.40"
1,7OYX,SNB,PZT,"0.66 , 0.17","39.78 , 27.51","0.61 , 0.27","17.72 , 13.41","0.69 , 0.63","4.42 , 2.33"
2,7OYX,SGB,MAG,"0.58 , 0.48","13.31 , 10.10","0.76 , 0.60","12.95 , 10.75","0.22 , 0.15","2.10 , 1.34"
3,7OYX,SGB,PZT,"3.62 , 0.85","41.73 , 27.69","2.87 , 2.33","40.92 , 38.11","0.73 , 0.00","1.90 , 0.00"
4,7OYX,MIXB,MAG,"0.10 , 0.08","5.26 , 2.66","0.11 , 0.15","4.48 , 3.69","0.11 , 0.06","3.11 , 1.53"
...,...,...,...,...,...,...,...,...,...
473,D4GQ,UAR,PZT,"0.31 , 0.04","1.97 , 0.77","0.26 , 0.17","3.55 , 1.48","0.17 , 0.05","0.74 , 0.27"
474,D4GQ,SE,MAG,"0.64 , 0.70","22.36 , 21.75","0.63 , 0.83","17.30 , 18.04","0.67 , 0.84","10.85 , 12.58"
475,D4GQ,SE,PZT,"0.80 , 0.71","8.07 , 3.08","0.51 , 0.70","14.73 , 20.78","0.23 , 0.13","1.80 , 1.15"
476,D4GQ,SS,MAG,"0.79 , 0.81","27.04 , 29.78","0.76 , 0.74","26.69 , 19.63","0.58 , 0.51","10.14 , 10.07"


In [9]:
bp_mag = rp_detection_all[rp_detection_all["Sensor"]=="MAG"]
bp_pzt = rp_detection_all[rp_detection_all["Sensor"]=="PZT"]

In [10]:
metric = "MAE Tb (s)"
normality_test(bp_mag[metric].str.split(" , ", expand=True).astype(float).iloc[:,0], sensor=metric, type="All data")
normality_test(bp_pzt[metric].str.split(" , ", expand=True).astype(float).iloc[:,0], sensor=metric, type="All data")

In [11]:
for metric in ["MAE Tb (s)", "MAE Ti (s)", "MAE Te (s)"]:
    bp_mag = rp_detection_all[rp_detection_all["Sensor"]=="MAG"][metric].str.split(" , ", expand=True).astype(float).iloc[:,0].values
    bp_pzt = rp_detection_all[rp_detection_all["Sensor"]=="PZT"][metric].str.split(" , ", expand=True).astype(float).iloc[:,0].values

    bp_both = pd.DataFrame({"MAG": bp_mag, "PZT": bp_pzt})
    dropped_values = len(bp_both) - len(bp_both.dropna())
    # if dropped_values > 0:
    #     print(f"removed {dropped_values} values due to NaNs")
    bp_both = bp_both.dropna()
    print(f'{metric}: {scipy.stats.wilcoxon(bp_both["MAG"].values, bp_both["PZT"].values)}')



MAE Tb (s): WilcoxonResult(statistic=4900.5, pvalue=2.2030222100029066e-15)
MAE Ti (s): WilcoxonResult(statistic=4420.0, pvalue=7.947332284509123e-18)
MAE Te (s): WilcoxonResult(statistic=3534.0, pvalue=1.6451559501530166e-20)


In [12]:
import numpy as np

In [13]:
bp_mag[metric].str.split(" , ", expand=True).astype(float).iloc[:,0].values

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
wilc = pd.DataFrame(columns=["Activity", "Metric", "Statistic", "p-value"])

for metric in ["MAE Tb (s)", "MAE Ti (s)", "MAE Te (s)"]:
    for activity in bp_mag["Activity"].unique():
        new_entry = {}
        new_entry["Activity"] = activity
        new_entry["Metric"] = metric
        new_entry["Statistic"], new_entry["p-value"] = scipy.stats.wilcoxon(bp_mag[bp_mag["Activity"]==activity][metric].str.split(" , ", expand=True).astype(float).iloc[:,0].values, bp_pzt[bp_pzt["Activity"]==activity][metric].str.split(" , ", expand=True).astype(float).iloc[:,0].values)
        wilc.loc[len(wilc)] = new_entry
        
wilc

In [ ]:
### Bias and variability analysis

In [14]:
for metric in ["tI (s)", "tE (s)", "tB (s)"]:
    for sensor in ["MAG", "PZT"]:
        bland_altman_plot(overview_all[sensor][metric], overview_all[sensor][metric.split(" ")[0] + " airflow " + metric.split(" ")[1]], metric=metric.split(" ")[0], sensor=sensor)
